I am going to continue off of my previous example of having multiple teachers from different subjects

## Imports

In [58]:
from agents import Agent, Runner, function_tool, trace
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display
import requests

In [59]:
load_dotenv(override=True)

True

In [60]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## Define Agents

In [61]:
math_instructions = "You are a 5th grade math teacher at a new school called: Memo Elementary School.\
    You are a generic 5th grade teacher but specialize in **Math**. Help the students the best you can."

science_instructions = "You are a 5th grade science teacher at a new school called: Memo Elementary School.\
    You are a generic 5th grade teacher but specialize in **Science**. Help the students the best you can."

humanities_instructions = "You are a 5th grade humanities teacher at a new school called: Memo Elementary School.\
    You are a generic 5th grade teacher but specialize in **Humanities**. Help the students the best you can."

In [62]:
math_agent = Agent (
    name="math_agent",
    instructions=math_instructions,
    model="gpt-4o-mini"
)

science_agent = Agent (
    name="science_agent",
    instructions=science_instructions,
    model="gpt-4o-mini"
)

humanities_agent = Agent (
    name="humanities_agent",
    instructions=humanities_instructions,
    model="gpt-4o-mini"
)

In [63]:
teacher_picker_instructions = """
You are a Principle that oversees an after-school tutoring program for 5th graders.

## Inputs
- Background information on student wanting a tutor
- Introductions from a few teachers that are willing to offer assistance 

## Output
Create a 2-3 sentence summary of what teacher you have chosen for the student and why
"""

In [64]:
teacher_picker_agent = Agent(
    name="teacher_picker_agent",
    instructions=teacher_picker_instructions,
    model="gpt-4o-mini"
)

## Agents as Tools

Let's consider an alternative where we turn each of the teachers into a tool that the teacher picker agent can use

In [65]:
math_tool = math_agent.as_tool(
    tool_name="math_teacher", 
    tool_description="General 5th grade teacher that specializes in Math"
)

science_tool = math_agent.as_tool(
    tool_name="science_teacher", 
    tool_description="General 5th grade teacher that specializes in Science"
)

humanities_tool = math_agent.as_tool(
    tool_name="humanities_teacher", 
    tool_description="General 5th grade teacher that specializes in Humanities"
)

teacher_picker_tool = teacher_picker_agent.as_tool(
    tool_name="teacher_picker",
    tool_description="Choose a teacher to tutor a student"
)

In [66]:
teacher_tools = [math_tool, science_tool, humanities_tool, teacher_picker_tool]

In [67]:
teacher_tools

[FunctionTool(name='math_teacher', description='General 5th grade teacher that specializes in Math', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'math_teacher_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000021CBA420C20>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='science_teacher', description='General 5th grade teacher that specializes in Science', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'science_teacher_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000021CBA582020>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_outp

## Handoffs

Handoff is kind of like giving agents information

Once we have a teacher chosen, we will send a text to the user as a handoff

Let's define tools for our handoff agent

In [68]:
subject_instructions = "You craft a short and brief subject line for a given message"
subject_agent = Agent(name="Subject Agent", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_agent.as_tool(
    tool_name="subject_tool",
    tool_description = "General email subject creator"
)

In [69]:
body_instructions = "You craft a short and brief message body based on information recieved"
body_agent = Agent(name="Body Agent", instructions=body_instructions, model="gpt-4o-mini")
body_tool = body_agent.as_tool(
    tool_name="body_tool",
    tool_description = "General email body creator"
)

In [ ]:
@function_tool
def send_message(subject: str, body: str) -> str:
    message = f"""
    {subject}
    
    {body}
    """
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [71]:
messenger_tools = [subject_tool,  body_tool, send_message]

In [72]:
messenger_tools

[FunctionTool(name='subject_tool', description='General email subject creator', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000021CBA4213A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='body_tool', description='General email body creator', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'body_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000021CBA583BA0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='send_message', desc

In [73]:
messenger_instructions = """You are a messager that is supposed to use a teacher's introduction and create/send a message to a student about offering after-school assistance \

Follow these steps exactly:    
1. Use the subject writer tool to generate a subject line for the message.
2. Use the body writer tool to generate a message body.
3. Use the send message tool to pass to send a message to the student. Pass the outputs from steps 1 and 2.
"""

In [74]:
messenger_agent = Agent(
    name="messenger_agent", 
    instructions=messenger_instructions, 
    model="gpt-4o-mini",
    tools=messenger_tools,
    handoff_description="Messenge student who their teacher is for after-school support"
)

In [75]:
handoffs = [messenger_agent]

## Define Main Agent

At this point we have the following...
- 3 Teacher Tools
- 1 Handoff Messenger Agent

In [76]:
instructions = """
You are a 5th grade Principle that delegates each student to the **best** teacher for an after-school tutoring program. 
You recieve background information on the student. 
Follow these steps exactly: 
1. Use the teacher tools to recieve background information on each of the teachers. 
2. Use the teacher generator tool to assign a teacher for the student. 
3. Pass ONLY the output from the teacher generator tool to the 'Messenger Agent' agent. 
The Messenger Agent will handle informing the student about the teacher they will be tutored from.
4. After sending, summarize what you sent in a friendly confirmation message.
"""

In [77]:
princple_agent = Agent(
    name="princple_agent",
    instructions=instructions,
    tools=teacher_tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

1. Running with `.run`

In [78]:
with trace("Teacher Picker w/ Tools"):
    result = await Runner.run(princple_agent, "I am a student struggling with Math")

In [81]:
display(result.final_output)

"The message offering after-school assistance in Math has been successfully sent to the student. If there's anything else you'd like me to do, please let me know!"

Running with `.run_streamed`

In [80]:
with trace("Teacher Picker using Push Tool"):
    result = Runner.run_streamed(princple_agent, "I am a student struggling with Math")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

The message offering after-school assistance in Math has been successfully sent to the student. If there's anything else you'd like me to do, please let me know!